In [ ]:
%matplotlib inline

모델 매개변수 최적화하기
==========================================================================

이제 모델과 데이터가 준비되었으니, 데이터에 대해서 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례이다.
모델을 학습하는 것은 *에폭(epoch)*이라고 부르는 반복적인 과정을 거친다. 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류 즉 손실(loss)을 계산하고,
매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤, 경사하강법을 사용하여 이 파라매터들을 **최적화(optimize)** 한다.
이 과정에 대한 자세한 설명은 `3Blue1Brown`의 [영상](<https://www.youtube.com/watch?v=tIeHLnjs5U8>)을 참고하라.

기본(Pre-requisite) 코드
------------------------------------------------------------------------------------------
이전 장에서 다룬 코드를 기져왔다.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



하이퍼파라매터(Hyperparameter)
------------------------------------------------------------------------------------------

하이퍼파라매터(Hyperparameter)는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다.
서로 다른 하이퍼파라매터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.
(참조: [하이퍼파라매터 튜닝(tuning)](<https://tutorials.pytorch.kr/beginner/hyperparameter_tuning_tutorial.html>))

학습 시에는 다음과 같은 하이퍼파라매터를 정의한다:
 - **에폭(epoch) 수** - 데이터셋을 반복하는 횟수
 - **배치 크기(batch size)** - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
 - **학습률(learning rate)** - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

최적화 단계(Optimization Loop)
------------------------------------------------------------------------------------------

하이퍼파라매터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다.
최적화 단계의 각 반복(iteration)을 **에폭**이라고 부른다.

하나의 에폭은 다음 두 부분으로 구성된다:
 - **학습 단계(train loop)** - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴한다.
 - **검증/테스트 단계(validation/test loop)** - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)한다.

학습 단계(training loop)에서 일어나는 몇 가지 개념들을 간략히 살펴보자. 






### 손실 함수(loss function)

학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다. **손실 함수(loss function)**는
획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 한다.
주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산한다.

일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 평균 제곱 오차(Mean Square Error) [`nn.MSELoss`](<https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss>), 
분류(classification)에 사용하는 음의 로그 우도(Negative Log Likelihood) [`nn.NLLLoss`](<https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss>),
그리고 ``nn.LogSoftmax``와 ``nn.NLLLoss``를 합친 [`nn.CrossEntropyLoss`](<https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss>) 등이 있다.

모델의 출력 로짓(logit)을 ``nn.CrossEntropyLoss``에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산한다.

In [ ]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

### 옵티마이저(Optimizer)

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다. **최적화 알고리즘**은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의한다.
모든 최적화 절차는 ``optimizer`` 객체에 캡슐화(encapsulate)된다. 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM이나 RMSProp과 같은 
[다양한 옵티마이저](<https://pytorch.org/docs/stable/optim.html>`)가 있다.

학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라매터를 등록하여 옵티마이저를 초기화한다.



In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 세단계로 이루어진다:
 * ``optimizer.zero_grad()``를 호출하여 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
 * ``loss.backwards()``를 호출하여 예측 손실(prediction loss)을 역전파한다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장한다.
 * 변화도를 계산한 뒤에는 ``optimizer.step()``을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.




전체 구현
------------------------------------------------------------------------------------------

최적화 코드를 반복하여 수행하는 ``train_loop``와 테스트 데이터로 모델의 성능을 측정하는 ``test_loop``를 정의하였다.



In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고 ``train_loop``와 ``test_loop``에 전달한다.
모델의 성능 향상을 알아보기 위해 자유롭게 에폭(epoch) 수를 증가시켜 볼 수 있다.



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

더 읽어보기
------------------------------------------------------------------------------------------

- [Loss Functions](<https://pytorch.org/docs/stable/nn.html#loss-functions>)
- [`torch.optim`](<https://pytorch.org/docs/stable/optim.html>)
- [Warmstart Training a Model](<https://tutorials.pytorch.kr/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html>)


